In [1]:
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
Setup complete. Using torch 2.1.0+cu121 (Tesla T4)


In [2]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="fdve01KrsgMvFjF1Sg5f")
project = rf.workspace("facemaskdetection-iwtpp").project("face_mask_detection-wvf80")
version = project.version(1)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...


In [4]:
!python train.py --img 416 --batch 4 --epochs 100 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

2024-03-08 10:30:01.546712: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 10:30:01.546792: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 10:30:01.548155: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/datasets/Face_Mask_Detection-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=4, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, mul

In [5]:
# source /path ---> images
# source 0 -----> webcam
# source /path of video
# source ---> url of youtube

!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source /content/mask.jpg

detect: weights=['runs/train/exp/weights/best.pt'], source=/content/mask.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-290-gb2ffe055 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/mask.jpg: 416x416 5 Masks, 1 No_Mask, 7.1ms
Speed: 0.4ms pre-process, 7.1ms inference, 487.0ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp13


In [16]:
from google.colab import output

def request_permission():
    # Request access to the webcam
    output.eval_js('''
        async function requestCam() {
          const stream = await navigator.mediaDevices.getUserMedia({ video: true });
          console.log("Permission granted");
          stream.getVideoTracks()[0].stop(); // Stop the stream immediately to release the webcam
        }
        requestCam();
    ''')
    print("Permission granted successfully")

request_permission()


Permission granted successfully


In [13]:
!pip install opencv-python-headless==4.5.4.60


In [19]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source 0


detect: weights=['runs/train/exp/weights/best.pt'], source=0, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-290-gb2ffe055 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

[ WARN:0] global /tmp/pip-req-build-4x5kub8r/opencv/modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video0): can't open camera by index
Traceback (most recent call last):
  File "/content/yolov5/detect.py", line 312, in <module>
    main(opt)
  File "/content/y

In [20]:
import cv2

def list_devices():
    num_devices = 0
    while True:
        cap = cv2.VideoCapture(num_devices)
        if not cap.isOpened():
            break
        print(f"Device {num_devices}: {cap.get(cv2.CAP_PROP_FRAME_WIDTH)}x{cap.get(cv2.CAP_PROP_FRAME_HEIGHT)}")
        cap.release()
        num_devices += 1

list_devices()
